In [2]:
import fnv
import fnv.reduce
import fnv.file  
import numpy as np
import seaborn as sns
import datetime as dt
import time
import pandas as pd
from tkinter import filedialog
from tkinter import *
import os
import matplotlib.pyplot as plt
import glob
import h5py
import cv2

h5py.run_tests()

...........s...........................................x...........................s.....s......................................x....................................sC:\Users\cloftus\Anaconda3\lib\site-packages\h5py\_hl\files.py:258: H5pyDeprecationWarning: File.fid has been deprecated. Use File.id instead.
  "Use File.id instead.", H5pyDeprecationWarning)
...s......ss.ss...............................................................................ss...................ssssss.....................................................................x....x.........................x......x..................................................ssss...................s........................................
----------------------------------------------------------------------
Ran 509 tests in 3.566s

OK (skipped=22, expected failures=6)


<unittest.runner.TextTestResult run=509 errors=0 failures=0>

In [3]:
os.chdir('C:/Users/cloftus/Desktop/')

In [4]:
options = np.array([000000, 200000, 400000, 600000, 800000, 1000000])

def round_to_fifth(time_):
    
    new = options[ abs(time_.microsecond - options).argmin() ]
    
    if new != 1000000:
        new_time = time_.replace(microsecond = new)
    else:
        temp_time = time_ + dt.timedelta(seconds = 1)
        new_time = temp_time.replace(microsecond = 000000)
    
    return(new_time)

def time_correct( _timestamp_ ):
    if drive == 'D:/thermal_data/':
        _timestamp_ = _timestamp_ - time_offset_D
    if drive == 'F:/':
        _timestamp_ = _timestamp_ - time_offset_F
        
    return(_timestamp_)

In [5]:
# root = Tk()
# root.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"),("all files","*.*")))
# print (root.filename)

In [6]:
datelist = [ d.date() for d in pd.date_range(start = pd.to_datetime('2019-08-06'), end = pd.to_datetime('2019-08-07')).to_pydatetime()]
print(datelist)

[datetime.date(2019, 8, 6), datetime.date(2019, 8, 7)]


In [ ]:
## writes tiff files, using all thermal videos every night, rounding timestamps, and synchronizing by frame number
# writes the files for the whole recordings (actually all of the recordings), but saves the tiff files in folders
# that chunk the tiff files into minute bins

zoom = 10 

options = np.array([000000, 200000, 400000, 600000, 800000, 1000000])

for date in datelist:
    
    curr_date = f'{date.year:04}' + f'{date.month:02}' + f'{date.day:02}'
    
    start_times = []
    end_times = []
    
    drives = ['D:/', 'E:/thermal_data/', 'F:/']
    for drive in drives:
        if os.path.isdir(drive + curr_date) == True:
            os.chdir(drive + curr_date)
            file_name = glob.glob('*.seq')
            print(file_name, flush = True)
            if len(file_name) > 0:
                im = fnv.file.ImagerFile(file_name[0])
                im.get_frame(0)
                day1 = int(im.frame_info[0]['value'][:3])
                start = f'{date.year:04}-' + f'{date.month:02}-' + f'{date.day:02} ' + im.frame_info[0]['value'][4:]
                start_dt = dt.datetime.strptime(start, '%Y-%m-%d %H:%M:%S.%f')
                start_seq = round_to_fifth(start_dt)
                start_times.append(start_seq)
                
                print( 'checkpoint 1', flush = True )

                im.get_frame(im.num_frames - 1)
                day2 = int(im.frame_info[0]['value'][:3])
                day_diff = day2 - day1
                end = f'{date.year:04}-' + f'{date.month:02}-' + f'{(date.day+day_diff):02} ' + im.frame_info[0]['value'][4:]
                end_dt = dt.datetime.strptime(end, '%Y-%m-%d %H:%M:%S.%f')
                end_seq = round_to_fifth(end_dt)
                end_times.append(end_seq)

                print( 'checkpoint 2', flush = True )
                
                if len(file_name) > 1:
                    wehaveaproblem
    
    if len(start_times) == 0:
        continue
    
    start_all = np.array(start_times).min()
    end_all = np.array(end_times).max()
    
    time_list = []
    
    print( 'checkpoint 3', flush = True )
    
    x = 0
    
    while x*200000 <= ((end_all - start_all).total_seconds())*1e6:
        time_list.append(start_all + dt.timedelta(microseconds=x*200000))
        x += 1
        
    print( 'checkpoint 4', flush = True )
        
    chunk_by = 300
        
    num_chunks = int((len(time_list)/chunk_by) + 1) # separate into minute chunks

    for drive in drives:
        
        if os.path.isdir(drive + curr_date) == True:
            os.chdir(drive + curr_date)
            file_name = glob.glob('*.seq')
            print(file_name, flush = True)
            if len(file_name) > 0:
                im = fnv.file.ImagerFile(file_name[0])
                
                os.chdir('C:/Users/cloftus/Desktop/Results/')
                
                chunk = 0
                
                im.get_frame(0)
                day1 = int(im.frame_info[0]['value'][:3])
                
                blank_mat = np.zeros((im.height, im.width))
                
                write_folder_name = drive[0] + "_" + time_list[chunk * chunk_by].strftime("%Y-%m-%d %H:%M:%S.%f").replace(' ', '_').replace('-', '').replace(':', '').replace('.', '')
                
                sub_time_list = time_list[ (chunk * chunk_by ) : ((chunk + 1) * chunk_by) ]
                
                os.makedirs( 'C:/Users/cloftus/Desktop/Results/' + write_folder_name )
                
                os.chdir( 'C:/Users/cloftus/Desktop/Results/' + write_folder_name )
                
                contains = []
                
                for i in range(im.num_frames):
                    
                    im.get_frame(i)
                    
                    im.unit = fnv.Unit.TEMPERATURE_FACTORY
                    
                    final = np.array(im.final, copy=False).reshape((im.height, im.width))
                    
                    day2 = int(im.frame_info[0]['value'][:3])
                    
                    day_diff = day2 - day1
                    
                    timestamp = f'{date.year:04}-' + f'{date.month:02}-' + f'{(date.day+day_diff):02} ' + im.frame_info[0]['value'][4:]
                    
                    timestamp = dt.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S.%f')
                    
                    if chunk != (num_chunks - 1):
                        
                        
                        while timestamp >= time_list[(chunk + 1) * (chunk_by)] :
                            
                            ## put an if i != 1 statement here if you don't want to write empty files at the beginning when one camera starts a little late
                            ## it's not entirely this easy. You have to handle how it's going to close the first folder. Or ma
                            
                            left_out = list(set( sub_time_list ) - set( contains ))
                            
                            for item in left_out:
                                
                                file_name = drive[0] + "_" + item.strftime("%Y-%m-%d %H:%M:%S.%f").replace(' ', '_').replace('-', '').replace(':', '').replace('.', '') + '.tiff'
           
                                fig, ax = plt.subplots(figsize = (blank_mat.shape[1] / (100/zoom), blank_mat.shape[0] / (100/zoom)))  

                                ax = sns.heatmap(blank_mat, xticklabels = False, yticklabels = False, cbar = False)
                
                                plt.savefig( file_name, bbox_inches = 'tight' )
                    
                                plt.close()
                                print(write_folder_name)
                                            
      
                            chunk += 1

                            write_folder_name = drive[0] + "_" + time_list[chunk * chunk_by].strftime("%Y-%m-%d %H:%M:%S.%f").replace(' ', '_').replace('-', '').replace(':', '').replace('.', '')
            
                            os.makedirs( 'C:/Users/cloftus/Desktop/Results/' + write_folder_name )

                            os.chdir( 'C:/Users/cloftus/Desktop/Results/' + write_folder_name )
                    
                            contains = []
                            
                            if chunk == (num_chunks - 1):
                                
                                print( 'I got to the end of chunks', flush = True )

                                sub_time_list = time_list[ (chunk * chunk_by ) : ]

                            else:
                                
                                sub_time_list = time_list[ (chunk * chunk_by ) : ((chunk + 1) * chunk_by) ]
                                

                    timestamp_rounded = sub_time_list[ np.argmin(abs(timestamp - np.array(sub_time_list))) ]  
                        
                    file_name = drive[0] + "_" + timestamp_rounded.strftime("%Y-%m-%d %H:%M:%S.%f").replace(' ', '_').replace('-', '').replace(':', '').replace('.', '') + '.tiff'
                    
                    final1 = final - final.mean()

                    fig, ax = plt.subplots(figsize = (final.shape[1] / (100/zoom), final.shape[0] / (100/zoom)))  

                    ax = sns.heatmap(final1, vmin = -np.std(final1), vmax = final1.max(), xticklabels = False, yticklabels = False, cbar = False)

                    plt.savefig( file_name , bbox_inches = 'tight')
                    
                    plt.close()

                    contains.append( timestamp_rounded )
                    
                    if i % 1000 == 0:
                        print(((i / im.num_frames)) * 100)
                
                left_out = list(set( sub_time_list ) - set( contains ))

                for item in left_out:
                    
                    
                    file_name = drive[0] + "_" + item.strftime("%Y-%m-%d %H:%M:%S.%f").replace(' ', '_').replace('-', '').replace(':', '').replace('.', '') + '.tiff'

                    fig, ax = plt.subplots(figsize = (blank_mat.shape[1] / (100/zoom), blank_mat.shape[0] / (100/zoom)))  

                    ax = sns.heatmap(blank_mat, xticklabels = False, yticklabels = False, cbar = False)

                    plt.savefig( file_name, bbox_inches = 'tight' )
                    
                    plt.close()
                    
                print( 'I got to the end', flush = True )


In [7]:
### helpful bit of code that renames files so they have a counter in front of the timestamp

root_path = 'C:/Users/cloftus/Desktop/Results/'

os.chdir(root_path)

folders = os.listdir()

for folder in folders:
        
        os.chdir(root_path + folder)
    
        files = os.listdir()
    
        counter = 0
        
        for file in files:
            
            if 'tiff' in file:
                #print( file.split( '_', 1) )
                #print( file.split( '_', 1)[1] )
                #os.rename( file,   file.split( '_', 1)[1] )
                #os.rename( file,   file[4: ] )
                #os.rename( file,  f'{counter:05}' + '_' + file )
                
                counter += 1

In [13]:
## makes a txt file for each folder in Results/ of all of the tiff images in that folder 
## this txt file will be used by ffmpeg in the command line to string the pictures together into a video

root_path = 'C:/Users/cloftus/Desktop/Results/'

os.chdir(root_path)

folders = os.listdir()

for folder in folders:
        
    os.chdir(root_path + folder)
    
    files = os.listdir()
    
    text_file = open(folder + '.txt', 'w')
        
    for file in files:

        if 'tiff' in file:

            n = text_file.write('file ' + "'" + file + "'" + '\n')

    text_file.close()

In [2]:
## writing picture folders into movies. This uses openCV and is an alternative to using ffmpeg

os.chdir('C:/Users/cloftus/Desktop/Results/')

folders = os.listdir()

for date in datelist:
    
    curr_date = f'{date.year:04}' + f'{date.month:02}' + f'{date.day:02}'
 
    for folder in folders:
        
        if curr_date in folder:
            
            os.chdir('C:/Users/cloftus/Desktop/Results/' + folder)

            images = [img for img in os.listdir() if img.endswith(".tiff")]
            
            print(images[0])
            
            frame = cv2.imread(images[0])
            height, width, layers = frame.shape

            video = cv2.VideoWriter(folder + '.avi', 0, 10, (width, height))

            for image in images:
                video.write(cv2.imread(image))

            cv2.destroyAllWindows()
            video.release()


        

NameError: name 'os' is not defined